# Вспомогательный код

Установка и импорт необходимых библиотек:

In [ ]:
!pip install -q gymnasium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 6.8 MB/s eta 0:00:00


In [ ]:
import torch
import random
import numpy as np
import gymnasium as gym

from torch import nn
from gym import Env, spaces
from itertools import product
from collections import deque
from IPython.display import clear_output

Чтобы результаты воспроизводились, зафиксируем seeds:

In [ ]:
def set_random_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


set_random_seed(42)
clear_output()

# Задание 1. Написание среды для игры в крестики-нолики

В этом задании вы должны:

- Реализовать собственную среду для игры в крестики нолики на основе `gymnasium.Env`, которая будет использоваться в других заданиях.
- Протестировать работу среды на примере игры двух случайных агентов.

## Формат результата
* Собственная работающая среда `TicTacToeEnv`.
* Играющие случайные агенты.
* Посчитанный винрейт для `'X'` для игры случайных агентов:

    ```
X wins in 58.53% games
    ```



## Среда для игры в крестики-нолики





**Описание среды:**

1. Игровое поле имеет размер 3×3, которое по ходу игры будет заполняться маркерами игроков `'X'` и `'0'`. В классе ниже отрисовка игрового поля реализована в методе `render`.

    Пример игрового поля:

    ```
0|X|0
_|X|0
X|_|_
    ```
2. Состояние среды отображается вектором (списком) из 9 чисел, в котором 0 обозначает незанятую ячейку, 1 — ячейку, занятую `'X'`, и −1 — ячейку, занятую `'0'`. Состояние среды хранится в атрибуте класса `self.cells`.

    Пример состояния среды для игрового поля выше:
    ```
[-1, 1, -1, 0, 1, -1, 1, 0, 0]
    ```
3. В атрибуте `self.player` хранится `X` или `0` — символ игрока, который сейчас ходит (меняется после каждого изменения среды).

4. В метод `step` передается `action` — номер ячейки, которую игрок хочет изменить. Агент может поставить соответствующий маркер только в незанятую ячейку посредством передачи номера ячейки в среду.

5. Игра заканчивается (`self.done = True`) в двух случаях: победа одного из игроков (проверяется методом `self.check_for_win`) или отсутствие пустых клеток на поле (проверяется методом `self.check_for_draw`).

6. Награда должна предоставляться за победу `'X'` в размере $+1$ очка, за победу `'0'`, соответственно, $-1$. В случае ничьей и нетерминальных состояний награда равна $0$.

Заполните недостающие фрагменты кода `# Your code here`. Помните, что пространство `self.action_space`, заполняемое игроками, [дискретно](https://gymnasium.farama.org/api/spaces/fundamental/#discrete).

In [ ]:
class TicTacToeEnv(Env):
    def __init__(self):
        # Define default variable
        self.cells = [0 for i in range(9)]  # environment state
        self.player = "X"  # current player (changes every step)
        self.done = False  # is the game over
        self.winner = None  # who is the winner

        # Symbols for rendering
        self.markers = {1: "X", 0: "_", -1: "0"}

        # Space https://gymnasium.farama.org/api/spaces/fundamental
        self.action_space = spaces.Discrete(9)  # Your code here

    def reset(self):
        """
        Bring game to initial state, define default variables.
        """
        # Your code here
        self.cells = [0 for i in range(9)]
        self.player = 'X'
        self.done = False
        self.winner = None


    def render(self):
        """
        Рrint game board.
        """
        cells = [self.markers[x] for x in self.cells]

        for j in range(0, 9, 3):
            print("|".join([cells[i] for i in range(j, j + 3)]))

    def legal_actions(self):
        """
        Check for actions available: check free cells
        """
        return [ind for ind, value in enumerate(self.cells) if value == 0]

    def check_for_win(self, cells):
        """
        Check that there is any win combination on the board.

        Parameters
        ----------
        cells: list
            Environment state

        Returns
        -------
            bool
            True if win, False in over cases
        """
        # Your code here
        if self.cells[0] == self.cells[1] == self.cells[2] != 0:
            return True
        if self.cells[3] == self.cells[4] == self.cells[5] != 0:
            return True
        if self.cells[6] == self.cells[7] == self.cells[8] != 0:
            return True
        if self.cells[0] == self.cells[3] == self.cells[6] != 0:
            return True
        if self.cells[1] == self.cells[4] == self.cells[7] != 0:
            return True
        if self.cells[2] == self.cells[5] == self.cells[8] != 0:
            return True
        if self.cells[0] == self.cells[4] == self.cells[8] != 0:
            return True
        if self.cells[2] == self.cells[4] == self.cells[6] != 0:
            return True
        else:
            return False
    def check_for_draw(self, cells):
        """
        Checking that the board is completely filled out.

        Parameters
        ----------
        cells: list
            Environment state

        Returns
        -------
            bool
            True if the board is completely filled out, False in over cases
        """
        if 0 not in cells:
            return True
        else:
            return False

    def step(self, action):
        """
        Player input process

        Parameters
        ----------
        action: int
            number of cell for change

        Returns
        -------
        observation: list
            New environment state
        reward: int
            Reward: 1 if win of 'X', -1 if win of '0', 0 in othrer cases
        self.done: bool
            Game over flag
        self.player: 'X' or '0'
            Player who takes the next step
        """
        # Check that action is possible
        assert self.action_space.contains(action), "impossible action"
        # Check that cell is empty
        assert (
            action in self.legal_actions()
        ), "not legal action"

        # Fill self.cells[action] depends on on whose turn (self.player) it is
        if self.player == "X":
            tmp = 1
        elif self.player == "0":
            tmp = -1
        self.cells[action] = tmp # Your code here

        observation = self.cells # Your code here

        # Check that there is any win combination on the board
        self.done = self.check_for_win(self.cells)  # Your code here

        if self.done and self.player == "X":
            reward = 1
            self.winner = "X"

        elif self.done and self.player == "0":
            reward = -1
            self.winner = "0"

        else:
            # Checking that the board is completely filled out
            self.done = self.check_for_draw(self.cells)
            reward = 0
            self.winner = None

        # toggle players
        if self.player == "X":
            self.player = "0"
        else:
            self.player = "X"

        return observation, reward, self.done, self.player

clear_output()

## Случайный агент

Реализуйте случайного агента, который выбирает действие случайным образом из доступных:

In [ ]:
class RandomAgent:
    def __init__(self, mark="X"):
        self.mark = mark

    def get_action(self, env):
        """
        Sample random LEGAL action from action space
        (use env.legal_actions and random.choice)

        Returns
        -------
        action: int
            number of cell for change
        """
        # Your code here
        action = np.random.choice(env.legal_actions())
        return action

In [ ]:
ttt = TicTacToeEnv()

x_agent = RandomAgent("X")
o_agent = RandomAgent("0")

rand_players = {"X": x_agent, "0": o_agent}

Визуализируйте игру между двумя случайными агентами:

In [ ]:
ttt.reset()

while not ttt.done:
    # which agent from `rand_players` is playing (use `ttt.player` info)
    player = ttt.player # Your code here
    # action of this agent
    action = rand_players[player].get_action(ttt) # Your code here
    # step
    state, reward, done, player = ttt.step(action) # Your code here
    ttt.render()
    print("\n")
print(f"{ttt.winner} wins! Reward is {reward}")

_|_|X
_|_|_
_|_|_


_|_|X
_|0|_
_|_|_


_|_|X
_|0|_
_|_|X


_|_|X
_|0|0
_|_|X


X|_|X
_|0|0
_|_|X


X|_|X
_|0|0
0|_|X


X|X|X
_|0|0
0|_|X


X wins! Reward is 1


## Винрейт для 'X' для случайных агентов

Давайте сделаем бейзлайн, с которым мы будем сравнивать результаты игры агентов, которых мы будем обучать. Для этого посчитаем, в каком проценте случаев `X` выигрывает на 100000 играх между случайными игроками, и дальше будем пробовать улучшить этот результат.

In [ ]:
wins = {"X": 0, "0": 0}

for i in range(100_000):
    ttt.reset()

    while not ttt.done:
        player = rand_players[ttt.player]
        action = player.get_action(ttt)

        state, reward, done, player = ttt.step(action)

    if ttt.winner is not None:
        wins[ttt.winner] += 1

print(f'X wins in {round((wins["X"]/100_000)*100, 2)}% games')

X wins in 58.39% games


# Задание 2. Обучение агента игре в крестики-нолики с помощью Q-learning

Создайте агента для игры в крестики-нолики и обучите его с помощью Q-learning, протестируйте жадного и $\varepsilon$-жадного агента.

## Формат результата

1. Обученные с помощью Q-learning агенты, игающие за `'X'`:
- жадный,
- $\varepsilon$-жадный.
2. Посчитанные для агентов винрейты при игре против случайного агента, играющего за `'0'`.

## Код Q-learning агента


В этой части задания вам необходимо реализовать Q-learning агента.




Вам необходимо будет заполнить `# Your code here`:
- метод `set_states`: нужно выбрать из множества комбинаций −1, 0 и 1 возможные состояния среды перед ходом `‘X’` (`‘X’` всегда ходит первым, поэтому количество 1 в состоянии среды должно быть равно количеству −1) и перед ходом `’0’` (количество 1 в состоянии среды должно быть на один больше количества −1).
- метод `set_Q_table`: нужно инициализировать случайными числами Q-значения всех легальных действий (легально заполнение пустых клеток) из этих состояний.
- метод `get_action`: нужно реализовать выбор лучшего или случайного действия в зависимости от типа агента и значения `epsilon`.
- метод `update_Q`: нужно реализовать основную формулу Q-learning.

Основная формула Q-learning:

$$Q(s,a) = Q(s,a)+α(R^a_{s} + \gamma\max_{a'}Q(s',a') -Q(s,a)),$$

где $s$ — состояние среды `state` в начале хода агента, $a$ — действие `action` агента на данном ходе, $Q(s,a)$ — значение Q-table `self.Q[current_state][action]` для состояния $s$ и действия $a$, $R^a_{s}$ — награда `reward` за действие `a` из состояния `s`, $s'$ — состояние среды после хода игрока и его оппонента, $a'$ — следующее действие игрока, $α$ — скорость обучения, $\gamma$ — дисконт за длинную игру.

**Совет:**
- При обновлении Q-значений (метод `update_Q`) учтите, что терминальное состояние игры не присутствует в Q-таблице (из него уже нельзя делать ход) и, соответственно, для состояний, предшествующих ему, $\max_{a'}Q(s',a')$ будет равно нулю.
- При победе `'X'` выдается награда $+1$, а в случае победы `'0'` — награда $-1$. Агент, играющий `'X',` должен выбирать действие с максимальным Q-значением, а `'0'` — с минимальным Q-значением.
- При обновлении Q-значений для действий из определенного состояния учтите, что $s'$ будет состоянием игрового поля не после хода игрока, а после хода оппонента.

In [ ]:
from itertools import product


class QTableAgent:
    def __init__(
            self, alpha=0.05, gamma=0.9, mark="X", epsilon=0.,
            epsilon_off=True
    ):
        """
        Parameters
        ----------
        alpha: float
            learning rate
        gamma: float
            discount coefficient
        mark: str
            'X' or '0' - player symbol
        epsilon: float
            epsilon for epsilon-greedy agent
        epsilon_off: boolean
            if True -'greedy' learning strategy or inference,
            if False 'epsilon-greedy' learning strategy
        """
        self.mark = mark
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_off = epsilon_off

        # Get possible for self.mark environment states
        self.states = self.set_states()
        # Init Q-table
        self.Q = self.set_Q_table()

    def set_states(self):
        """
        Set possible for self.mark environment states

        Returns
        -------
        states: set
            Set of possible for self.mark environment states
        """
        # Set of all possible marker compositions
        states = set(product(*[list(range(-1, 2)) for _ in range(9)]))
        # Subset of states for X player
        # contains states in which both players took equal number of actions
        # (since X goes first)
        if self.mark == "X": # select with condition
            states = [i for i in states if i.count(-1) == i.count(1)] # Your code here

        # Subset of states for 0 player
        # contains states in which X player took one more action than 0 player
        # (since 0 goes second)

        elif self.mark == "0": # select with condition
            states = [i for i in states if i.count(-1) == (i.count(1) + 1)] # Your code here
        return states

    def set_Q_table(self):
        """
        Init Q-table.

        Returns
        -------
        Q: dict
            Q-table[state][action] for possible states and actions with
            random gauss (mean=0, sigma=0.1)
        """
        Q = {}
        # Match legal actions for each possible action in each state with
        # random initial number
        for state in self.states:
            Q[state] = {}
            # Your code here
            for i in range(len(state)):
                if state[i] == 0:
                    Q[state][i] = np.random.rand() # Your code here
        return Q

    def get_action(self, env):
        """
        Sample optimal or random action.

        Parameters
        ----------
        env: TicTacToeEnv
            environment

        Returns
        -------
        action: int
            number of cell for change
        """
        state = tuple(env.cells)
        rand = random.uniform(0, 1)

        if self.epsilon_off or rand >= self.epsilon:
            # Sample optimal action (based on greediness)
            if self.mark == "X":
                action = max(set(self.Q[state].keys()) & set(env.legal_actions())) # Your code here
            else:
                action = min(set(self.Q[state].keys()) & set(env.legal_actions()))# Your code here
        else:
            # Sample random  action
            action = np.random.choice(env.legal_actions())# Your code here

        return action

    def update_Q(self, current_state, action, next_state, reward, done):
        """
        Q-table update.

        Parameters
        ----------
        current_state: list
            Current environment state
        action: int
            Current agent action
        next_state: list
            Environment state after agent action and opponent action
        reward: int
            Reward
        done: bool
            Game over flag
        """
        current_state = tuple(current_state)
        if not done:
            next_state = tuple(next_state)
            next_state_value = max(self.Q[next_state], key=self.Q[next_state].get)
        else:
            next_state_value = 0

        # Q-learning update folmula
        self.Q[current_state][action] += self.alpha * (reward + self.gamma * next_state_value - (self.Q[current_state][action]))# Your code here

## Обучение жадного агента `'X'`:

Обучите агента, играющего за `X`, придерживающегося жадной стратегии, против **случайного** агента, играющего за `0`, на 1 миллионе игр и сравните их винрейты между собой и с бейзлайном.

**Совет:**
- При обучении учтите, что $s'$ будет состоянием игрового поля не после хода игрока, а после хода оппонента.
- Помните, что `list` в python является [изменяемым типом данных](https://realpython.com/python-mutable-vs-immutable-types/). Используйте `.copy()` чтобы изолировать состояние среды, подаваемое агенту.

In [ ]:
ttt = TicTacToeEnv()

x_agent = QTableAgent(0.05, 0.9, "X", epsilon=0.1, epsilon_off=True)

o_agent = RandomAgent("0")

players = {"X": x_agent, "0": o_agent}

In [ ]:
for i in range(1_000_000):
    ttt.reset()
    while not ttt.done:
        player = players[ttt.player]
        if player.mark == "X":
            action_x = player.get_action(ttt)  # Your code here
            current_state = ttt.cells.copy() # Your code here
            state, reward, done, _ = ttt.step(action_x) # Your code here
            if done:

                players["X"].update_Q(current_state, action_x, None, reward, done) # Your code here
        else:
            action_o = player.get_action(ttt) # Your code here
            next_state, reward, done, _ = ttt.step(action_o) # Your code here

            players["X"].update_Q(current_state, action_x, next_state, reward, done) # Your code here

Посмотрим на винрейт:

In [ ]:
wins = {"X": 0, "0": 0}

for i in range(100_000):
    ttt.reset()

    while not ttt.done:
        player = players[ttt.player]
        action = player.get_action(ttt)

        state, reward, done, player = ttt.step(action)

    if ttt.winner is not None:
        wins[ttt.winner] += 1

print(f'X wins in {round((wins["X"]/100000)*100, 2)}% games')

X wins in 78.14% games


## Обучение $\varepsilon$-жадного агента `'X'`:

Обучите агента, играющего за `X`, придерживающегося $\varepsilon$-жадной стратегии, против **случайного** агента, играющего за `0`, на 1 миллионе игр и сравните их винрейты между собой и с бейзлайном.

In [ ]:
ttt = TicTacToeEnv()

x_agent = QTableAgent(0.05, 0.9, "X", epsilon=0.1, epsilon_off=False)
o_agent = RandomAgent("0")

players = {"X": x_agent, "0": o_agent}

In [ ]:
for i in range(1_000_000):
    ttt.reset()
    while not ttt.done:
        player = players[ttt.player]
        if player.mark == "X":
            action_x = player.get_action(ttt) # Your code here
            current_state = ttt.cells.copy() # Your code here
            state, reward, done, _ = ttt.step(action_x) # Your code here
            if done:
                players["X"].update_Q(current_state, action_x, None, reward, done) # Your code here
        else:
            action_o = player.get_action(ttt)# Your code here
            next_state, reward, done, _ = ttt.step(action_o) # Your code here

            players["X"].update_Q(current_state, action_x, next_state, reward, done) # Your code here

Посмотрим на винрейт. Обратите внимание на то, что мы **выключаем примешивание случайных действий при тестировании модели**. Без этого качество модели будет сильно занижено!

In [ ]:
wins = {"X": 0, "0": 0}
players["X"].epsilon_off = True

for i in range(100_000):
    ttt.reset()

    while not ttt.done:
        player = players[ttt.player]
        action = player.get_action(ttt)

        state, reward, done, player = ttt.step(action)

    if ttt.winner is not None:
        wins[ttt.winner] += 1

print(f'X wins in {round((wins["X"]/100_000)*100, 2)}% games')

X wins in 78.09% games


## Попробуйте сразиться с вашим обученным агентом

Можете попробовать сразиться с вашим обученным агентом.
В качестве ввода можете использовать либо ручной ввод, либо случайный:

*   `human_action = int(input())  # manual input`
*   `human_action = np.random.choice(ttt.legal_actions())`


In [ ]:
ttt.reset()

while not ttt.done:
    ttt.render()
    print("\n")
    print(f"It's {ttt.player} turn")
    if ttt.player == "X":
        action = players["X"].get_action(ttt)
        state, reward, done, player = ttt.step(action)

        ttt.render()
        print("\n")
        continue
    else:
        print(f"chose action: {ttt.legal_actions()}")
        human_action = int(input())  # manual input
        #human_action = np.random.choice(ttt.legal_actions())
        state, reward, done, player = ttt.step(human_action)

        ttt.render()
        print("\n")

print(f"{ttt.winner} wins! Reward is {reward}")

_|_|_
_|_|_
_|_|_


It's X turn
_|_|_
_|_|_
_|_|X


_|_|_
_|_|_
_|_|X


It's 0 turn
chose action: [0, 1, 2, 3, 4, 5, 6, 7]
4
_|_|_
_|0|_
_|_|X


_|_|_
_|0|_
_|_|X


It's X turn
_|_|_
_|0|_
_|X|X


_|_|_
_|0|_
_|X|X


It's 0 turn
chose action: [0, 1, 2, 3, 5, 6]
6
_|_|_
_|0|_
0|X|X


_|_|_
_|0|_
0|X|X


It's X turn
_|_|_
_|0|X
0|X|X


_|_|_
_|0|X
0|X|X


It's 0 turn
chose action: [0, 1, 2, 3]
2
_|_|0
_|0|X
0|X|X


0 wins! Reward is -1


**Вывод: я победил!**
При обучении жадного и эпсилон-жадного агента получили сопоставимое качество. Возможно, получится еще немного улучшить результат при подборе эпсилон.

# Задание 3. Обучение агента игре в крестики-нолики при помощи DQN

Обучите с помощью Deep Q-learning агента игре в крестики-нолики.

## Формат результата
Обученный с помощью Deep Q-learning агент, игающий за `'X'`, с винрейтом против случайного агента, играющего за `'0'`, ~85%.

## Код DQN агента

Для хранения experience replay будем использовать [deque](https://proproprogs.ru/structure_data/std-ochered-collectionsdeque-na-python).

В этой части задания вам необходимо реализовать DQN агента.




Вам необходимо будет заполнить `# Your code here`:
- метод `set_Q_network`: нужно реализовать архитектуру сети. Архитектура сети должна быть устроена следующим образом: на вход принимаются 9 значений, которые соответствуют состоянию игрового поля, на выход выдаются 9 значений, которые соответствуют Q-значениям для 9 возможных действий, скрытые слои — на ваш выбор, поэкспериментируйте с разными архитектурами.
- метод `get_action`: нужно реализовать выбор лучшего или случайного разрешенного действия в зависимости от типа агента и значения `epsilon`. Для получения списка разрешенных действий используйте `env.legal_actions()`.
- метод `update_target_network`: необходимо копировать параметры сети `self.Q_net`(используется для обучения) в сеть `self.target_net` (используется для предсказания Q-значений для следующго состояния $s'$). Можно использовать `state_dict()`.

<img src="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/Exercises/EX15/dqn_loss.png" alt="Drawing" width="800"/>

**Совет:** обратите внимание, что не все 9 значений будут доступны для определенных состояний, учтите это в методе `get_action`.

In [ ]:
from collections import deque


class DQNAgent(nn.Module):
    def __init__(
        self, gamma=0.9, mark="X", memory_size=10000, epsilon=0.,
        epsilon_off=True
    ):
        """
        Parameters
        ----------
        alpha: float
            learning rate
        gamma: float
            discount coefficient
        mark: str
            'X' or '0' - player symbol
        memory_size: int
            size of memory buffer
        epsilon: float
            epsilon for epsilon-greedy agent
        epsilon_off: boole
            if True -'greedy' learning strategy or inference,
            if False 'epsilon-greedy' learning strategy
        """
        super(DQNAgent, self).__init__()
        self.mark = mark
        self.gamma = torch.tensor(gamma, dtype=float)
        self.epsilon = epsilon
        self.epsilon_off = epsilon_off

        # Experience replay
        self.exp_replay = deque(maxlen=memory_size)
        # Q-Network (for learning and Q(s, a))
        self.Q_net = self.set_Q_network()
        # Target-Network (for Q(s', a'))
        self.target_net = self.set_Q_network()
        self.update_target_network()

    def set_Q_network(self):
        """Set Q_net architecture.

        Returns
        -------
        Q_net: nn.Sequential
            Q_net architecture
        """
        Q_net = nn.Sequential(
            nn.Linear(9, 27),
            nn.ReLU(),
            nn.Linear(27, 81),
            nn.ReLU(),
            nn.Linear(81, 27),
            nn.ReLU(),
            nn.Linear(27, 9)) # Your code here
        return Q_net

    def forward(self, states):
        """Forward pass.

        Parameters
        ----------
        states: list, np.array or torch.Tensor [batch_size, 9]
            batch of environment states at the beginning of the agent's action

        Returns
        -------
        Q_vals: torch.Tensor [batch_size, 9]
            Q-vals for all 9 action (not all of this action legal)
        """
        states = torch.Tensor(states)
        Q_vals = self.Q_net(states)
        return Q_vals

    def get_action(self, Q_vals, env):
        """
        Sample optimal or random legal action.

        Parameters
        ----------
        Q_vals: torch.Tensor [batch_size, 9]
            Q-vals for all 9 action (not all of this action legal)
        env: TicTacToeEnv
            environment

        Returns
        -------
        action: int
            number of cell for change
        """
        state = torch.Tensor(env.cells)
        # Get legal action from env, transform to torch.int64 tensor

        legal_actions = torch.LongTensor(env.legal_actions()) # Your code here

        index = torch.zeros(9, dtype=bool)
        index[legal_actions.to(torch.int64)] = True
        rand = random.uniform(0, 1)
        if self.epsilon_off or rand >= self.epsilon:
            # Sample optimal action
            if self.mark == "X":
                best_q = max(torch.masked_select(Q_vals, index)) # Your code here
            else:
                best_q = min(torch.masked_select(Q_vals, index)) # Your code here
            action = torch.logical_and(Q_vals == best_q, index).nonzero()[0].item()

        else:
            # Sample random action
            action = np.random.choice(legal_actions) # Your code here

        return int(action)

    def add_to_memory(self, current_state, action, next_state, reward, done):
        """Add data to experience replay.

        Parameters
        ----------
        current_state: torch.Tensor [batch_size, 9]
            Current environment state
        action: int
            Current agent action
        next_state: torch.Tensor [batch_size, 9]
            Environment state after agent action and opponent action
        reward: int
            Reward
        done: bool
            Game over flag
        """
        self.exp_replay.append((current_state, action, next_state, reward, done))

    def update_target_network(self):
        """Use Q_net parameters to update target_net."""
        # Your code here
        self.target_net.load_state_dict(self.Q_net.state_dict())

## Код обучения

Реализуйте функцию, рассчитывающую TD loss:
$$ L = { 1 \over N} \sum_i [ Q_{\theta}(s,a) - Q_{reference}(s,a) ] ^2 $$

С Q-reference, определенным как:

$$ \large Q_{reference}(s,a) = R^a_{s} + \gamma \cdot \max_{a'} Q_{target}(s', a'), $$

где:
* $R^a_{s}$ — награда `reward` за действие `a` из состояния `s`,
* $Q_{target}(s',a')$ — $Q$-значение следующего состояния системы `next_states` и следующего действия, вычисленное `agent.target_net`,
* $s, a, s'$ — текущее состояние `states`, действие `actions` и следующее состояние `next_states`,
* $\gamma$ — коэффициент дисконтирования `agent.gamma`.

**Совет:**
- При расчете $Q_{reference}(s,a)$ `reference_q` учтите, что терминальное состояние игры не присутствует в Q-таблице (из него уже нельзя делать ход) и, соответственно, для состояний, предшествующих ему, $\max_{a'} Q_{target}(s', a')$ будет равно нулю (используйте `is_not_done`).

In [ ]:
def compute_td_loss(batch, agent):
    states = torch.tensor(np.array([x[0] for x in batch]), dtype=torch.float) # 9
    actions = torch.tensor(np.array([x[1] for x in batch]), dtype=torch.int64) # 1
    next_states = torch.tensor(np.array([x[2] for x in batch]), dtype=torch.float) # 9
    rewards = torch.tensor(np.array([x[3] for x in batch]), dtype=torch.int64) # 1
    is_not_done = 1 - torch.tensor(np.array([x[4] for x in batch]), dtype=torch.int64) # bool

    q_vals = agent(states) # Your code here
    current_q = q_vals[range(len(actions)), actions] # Your code here
    target_q = agent.target_net(next_states)  # Your code here
    reference_q = rewards + agent.gamma * target_q.max(axis=-1)[0] * is_not_done # Your code here
    loss = torch.mean((current_q - reference_q.detach()) ** 2) # Your code here

    return loss

Реализуем функцию, выдающую винрейт игрока на 10к играх (не забываем выключать примешивание случайных действий при тестировании модели):

In [ ]:
def get_winrate():
    wins = {"X": 0, "0": 0}
    players["X"].epsilon_off = True

    for i in range(10_000):
        ttt.reset()

        while not ttt.done:
            player = players[ttt.player]
            if player.mark == "0":
                action = player.get_action(ttt)
            else:
                q_vals = player(ttt.cells)
                action = player.get_action(q_vals, ttt)

            state, reward, done, player = ttt.step(action)

        if ttt.winner is not None:
            wins[ttt.winner] += 1
    wr = (wins["X"] / 10000) * 100
    print(f"X wins in {round(wr, 2)}% games")
    return wr

In [ ]:
target_wr = 85.0

ttt = TicTacToeEnv()

x_agent = DQNAgent(gamma=0.8, mark="X", epsilon=0.2, epsilon_off=False)
o_agent = RandomAgent("0")

players = {"X": x_agent, "0": o_agent}

opt = torch.optim.Adam(players["X"].Q_net.parameters())

Далее мы обучим игрока `X` против случайного игрока `0`. Задача — добиться винрейта ~85%.

Процесс обучения строится следующим образом:
* во время одного эпизода обучения на протяжении 50 игр собирается игровой опыт в буфер памяти агента, а именно текущее состояние, действие из него, следующее состояние, награда и флаг конца эпизода;
* из памяти выбирается случайный батч и обновляются веса Q-сети;
* после каждых 50 подобных эпизодов обновляются веса target network, следите за изменением винрейта после каждых 500 эпизодов.

Вам необходимо заполнить пропуск `# Your code here`, в котором заполняется буфер памяти агента.

**Совет:**
- При обучении учтите, что $s'$ будет состоянием игрового поля не после хода игрока, а после хода оппонента.
- Обучение RL-модели неустойчиво. Небольшое изменение архитектуры сети или даже [выбор seed](https://www.alexirpan.com/2018/02/14/rl-hard.html) может существенно поменять результат.

In [ ]:
batch[0]

([1, 0, 0, 0, -1, 0, -1, 0, 1], 2, [1, 0, 1, -1, -1, 1, -1, 0, 1], 0, False)

In [ ]:
clear_output()

for j in range(10_000):
    for i in range(50):
        # Your code here
        ttt.reset()
        while not ttt.done:

            player = players[ttt.player]

            if player.mark == 'X':
                q_vals = player(ttt.cells.copy())
                action_x = player.get_action(q_vals, ttt)
                current_state = ttt.cells.copy()
                state, reward, done, _ = ttt.step(action_x)

                if done:
                    players['X'].add_to_memory(current_state, action_x, next_state, reward, done)
            else:
                action_o = player.get_action(ttt)
                next_state, reward, done, _ = ttt.step(action_o)

                players['X'].add_to_memory(current_state, action_x, next_state, reward, done)

    batch = random.sample(players["X"].exp_replay, 128)

    opt.zero_grad()
    loss = compute_td_loss(batch, players["X"])
    loss.backward()
    nn.utils.clip_grad_norm_(players["X"].Q_net.parameters(), 1.0)
    opt.step()

    if j % 50 == 0:
        players["X"].update_target_network()

    if j % 500 == 0:
        wr = get_winrate()

    if wr > target_wr:
        break
        if players["X"].epsilon > 0:
            players["X"].epsilon -= 0.005

X wins in 80.15% games
X wins in 77.53% games
X wins in 78.75% games
X wins in 81.12% games
X wins in 83.8% games
X wins in 86.14% games
